In [1]:
##Our import: 
import nltk
from nltk.probability import ConditionalFreqDist
import pandas as pd
from pandas import DataFrame
import math
from nltk import word_tokenize
import numpy as np
from random import randint
import csv
import numpy
import re

import sys

from sklearn.cross_validation import KFold
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.feature_extraction import DictVectorizer

%matplotlib inline
import matplotlib.pyplot as plt
import csv
from textblob import TextBlob
import pandas
import sklearn
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import StratifiedKFold, cross_val_score, train_test_split 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.learning_curve import learning_curve


##Our defines: 
q1Verbose=1
q113_verbose=0
q2Verbose=1

### Q1. Document Classification
#### Q1.1. Reuters Dataset

##### Q1.1.1 Turn the code of the Sklearn tutorial above into a notebook.

This code is taken from the out of core classification guide given in the assigmnent. 
http://scikit-learn.org/dev/auto_examples/applications/plot_out_of_core_classification.html#example-applications-plot-out-of-core-classification-py


In [9]:
# Notebook attached in folder, in order to avoid clattering. 
# Taken from import example-application-plot-out-of-core-classification.pt so we could call it's functions
from plot_out_of_core_classification import *

##### Q1.1.2 Explore how many documents are in the dataset, how many categories, how many documents per categories, provide mean and standard deviation, min and max. (Hint: use the pandas library to explore the dataset, use the dataframe.describe() method.)


In [10]:
#From the code, accessing the reuters document data base. 
data_stream = stream_reuters_documents()

#Experimenting with functions given in hint. 
df = pd.DataFrame(data_stream)
print("The type of df is: ", type(df))
#df

The type of df is:  <class 'pandas.core.frame.DataFrame'>


#### How many documents in the dataset? 

In [11]:
df['title'].describe()['count']


21578

Output: 21578

There are 21,578 as seen below. 

In [ ]:
#Some exploring of dataset: 
if q1Verbose:
    print(list(df))
    print(df['body'].describe())
    # Since df['topics'].descibe() does not work, let's just print. 
    print(df['topics'])
    print(type(df['topics']))

#### Each document can belong to a few topics (we interperted topics for categories). We will feed all information into a FreqDist and receive statistics.

As implied, this means a document can appear in several documents. 

In [13]:
#Create a list of all occurences of all topics and feed to FreqDist. 
freq_dist = nltk.FreqDist(sum(list(df['topics']), []))

#### How many categories: 

In [14]:
category_set=set(sum(list(df['topics']), []))
num_of_categories = len(category_set)
print("The number of categories is: ",num_of_categories)

The number of categories is:  445


Output: "The number of categories is:  445"
Since we are using a Frequency distribution, the number of categories can also be retrieved by: 

In [18]:
len(freq_dist)

445

#### How many documents per category: 
Since we are using a Frequency distribution, the number of documents per category 
is the value in freq_dist[category]. 

We can print a list of all categories and the number of documents in them. 

In [19]:
#Create a list of (Category, Number of doc's)
cat_numOfDocs = [(category, freq_dist[category]) for category in category_set]
for pair in cat_numOfDocs[:10]: 
    print('Category: ', pair[0], 'has ', pair[1], 'Docs')

Category:  naphtha has  7 Docs
Category:  uganda has  17 Docs
Category:  brunei has  6 Docs
Category:  coconut has  6 Docs
Category:  oapec has  2 Docs
Category:  austdlr has  4 Docs
Category:  antigua has  1 Docs
Category:  gnp has  163 Docs
Category:  palm-oil has  43 Docs
Category:  lit has  3 Docs


Output:
Category:  naphtha has  7 Docs
Category:  uganda has  17 Docs
Category:  brunei has  6 Docs
Category:  coconut has  6 Docs
Category:  oapec has  2 Docs
Category:  austdlr has  4 Docs
Category:  antigua has  1 Docs
Category:  gnp has  163 Docs
Category:  palm-oil has  43 Docs
Category:  lit has  3 Docs

####  Provide mean and standard deviation, min and max. 
Mean: Mean number of documents per categorie. 

Not for the last time during this assignment, We didn't find the "leads" in the attached notebooks, 
ended up working hard, and at a later stage, noticing the lead when researching a different problem. 

In [17]:
#Obviously We first went the wrong way and worked very hard. We will first present the elegant way. 
#Simply add a length column, and describe it. 
df['length'] = df['body'].map(lambda text: len(text))
df['length'].describe()

count    21578.000000
mean       718.527621
std        819.414178
min          0.000000
25%        235.000000
50%        491.000000
75%        904.000000
max       9875.000000
Name: length, dtype: float64

In [20]:
#Some insight into the actual process: 

#Mean: 
#Sum of number of documents per each category. 
sum_docs_cat = sum(num_of_docs for (cat, num_of_docs) in cat_numOfDocs)
#
#Mean expected number of documents per categorie. 
#mean_exp=sum(freq_dist.freq(cat)*num_of_docs for (cat, num_of_docs) in cat_numOfDocs)
#mean2 = np.mean([num_of_docs for _, num_of_docs in cat_numOfDocs])
#print('The Mean number of documents per categorie is: ', mean_exp)
#print('The Mean2 number of documents per categorie is: ', mean2)
mean = sum_docs_cat/len(cat_numOfDocs)
print('The Mean (Average) number of documents per category is: ', mean)

#Max:
print('The category with maximum documents is: "',freq_dist.max(), '"which has ', freq_dist[freq_dist.max()], ' documents.')

#Min:
min_num_of_docs = sorted(cat_numOfDocs ,key=lambda x: x[1])[0][1]
cats_w_min_num_of_docs = [cat for (cat, num_of_docs) in cat_numOfDocs if num_of_docs==min_num_of_docs]
display = 3 #Display only part of categories, not all. 
print('The category with minimum documents are:',cats_w_min_num_of_docs[:display], 'who have', min_num_of_docs, 'documents each. ' )

#Standard deviation: 
std_dev = math.sqrt(sum( (math.pow(num_of_docs-mean, 2) for (_, num_of_docs) in cat_numOfDocs))/len(cat_numOfDocs))
print('The standard deviation in number of documents per category is:', std_dev)

The Mean (Average) number of documents per category is:  89.87191011235954
The category with maximum documents is: " usa "which has  12542  documents.
The category with minimum documents are: ['antigua', 'bhutan', 'corn-oil'] who have 1 documents each. 
The standard deviation in number of documents per category is: 643.9321684195975


The Mean (Average) number of documents per category is:  89.87191011235954
The category with maximum documents is: " usa "which has  12542  documents.
The category with minimum documents are: ['antigua', 'bhutan', 'corn-oil'] who have 1 documents each. 
The standard deviation in number of documents per category is: 643.9321684195975

#### Q1.1.3 Explore how many characters and words are present in the documents of the dataset.

First we consider all diffferent word tokens and characters in code, as in a set of elements. We then calculate 
the number of all tokens and characters all together, which is more relevent to our issues. 

In [ ]:
#Create sets of words and characters. 
#Takes a while to run, use with care :)
if q113_verbose: 
    word_set=set()
    word_list=[]
    for i in range(len(df['body'])): 
        word_set.update(word_tokenize(df['body'][i]))
        word_list += word_tokenize(df['body'][i])

    char_set=set()
    char_list=[]
    for word in word_set: 
        for letter in word: 
            char_set.update(letter)
            char_list += letter
    print('There are %d different words in all documents. ' %len(word_set))
    print('There are %d word tokens in all documents. ' %len(word_list))
    print('There are %d different characters in all documents. ' %len(char_set))
    print('There are %d characters in all documents. ' %len(char_list))


Since runtime is long for the two above boxes, Output given here, no need to run. 

Output: 
There are 76886 different words in all documents. 
There are 2854622 word tokens in all documents. 
There are 89 different characters in all documents. 
There are 568599 characters in all documents. 


We will now construct a dictionary, That maps from article index to {num_of_words: , num_of_chars: }

In [23]:
article_2words_chars = {}
for i in range(len(df['body'])): 
    article_2words_chars[i] = (len(word_tokenize(df['body'][i])), len(df['body'][i]))

In [27]:
def explore_doc(x):
    print('Document with index %d has %d words and %d letters' % (x, article_2words_chars[x][0], article_2words_chars[x][1]))

In [32]:
#Do some exploring: (We'll print statistic's for some random doc's.)
i=0
while i<5: 
    x = randint(0,len(df['body']))
    explore_doc(x)
    i+=1

Document with index 1051 has 111 words and 572 letters
Document with index 7507 has 444 words and 2593 letters
Document with index 18162 has 299 words and 1667 letters
Document with index 8530 has 102 words and 533 letters
Document with index 12794 has 25 words and 135 letters


Document with index 1051 has 111 words and 572 letters
Document with index 7507 has 444 words and 2593 letters
Document with index 18162 has 299 words and 1667 letters
Document with index 8530 has 102 words and 533 letters
Document with index 12794 has 25 words and 135 letters

#### Q1.1.4 Explain informally what are the classifiers that support the "partial-fit" method discussed in the code.

Informally, the classifiers that support "partial-fit", are classifiers who do not need to "hold" all the
information they are given, at every given moment. If we attempt a slightly more formal explanation, We 
can say that the state of the classifier is changed as it learns from more inputs, yet this input is not
a state variable. 

#### Q1.1.5 Explain what is the hashing vectorizer used in this tutorial.
####            Why is it important to use this vectorizer to achieve "streaming classification"?

The Hashing Vectorizer used in this tutorial is a tool for counting words as features, with a few tweaks 
from the "usual" count vectors. It is first given a size (in our case, 2^18, ~num of words), and later, 
while meeting words, it will add them to the appropriate cell, using a hash function. 

By using the hashing vectorizer, We achieve two main advantages. 
1) Hashing strings into integers results in imporved memory usage. 
2) Our feature space remains the same over different batches. 

### Q1.2 Spam Dataset

In [4]:
import os
import numpy
from pandas import DataFrame
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.cross_validation import KFold
from sklearn.metrics import confusion_matrix, f1_score

NEWLINE = '\n'

HAM = 'ham'
SPAM = 'spam'

SOURCES = [
    ('data/spam',        SPAM),
    ('data/easy_ham',    HAM),
    ('data/hard_ham',    HAM),
    ('data/beck-s',      HAM),
    ('data/farmer-d',    HAM),
    ('data/kaminski-v',  HAM),
    ('data/kitchen-l',   HAM),
    ('data/lokay-m',     HAM),
    ('data/williams-w3', HAM),
    ('data/BG',          SPAM),
    ('data/GP',          SPAM),
    ('data/SH',          SPAM)
]

SKIP_FILES = {'cmds'}

In [5]:

def read_files(path):
    for root, dir_names, file_names in os.walk(path):
        for path in dir_names:
            read_files(os.path.join(root, path))
        for file_name in file_names:
            if file_name not in SKIP_FILES:
                file_path = os.path.join(root, file_name)
                if os.path.isfile(file_path):
                    past_header, lines = False, []
                    f = open(file_path, encoding="latin-1")
                    for line in f:
                        if past_header:
                            lines.append(line)
                        elif line == NEWLINE:
                            past_header = True
                    f.close()
                    content = NEWLINE.join(lines)
                    yield file_path, content


In [6]:
def build_data_frame(path, classification):
    rows = []
    index = []
    for file_name, text in read_files(path):
        rows.append({'text': text, 'class': classification})
        index.append(file_name)

    data_frame = DataFrame(rows, index=index)
    return data_frame

In [7]:
data = DataFrame({'text': [], 'class': []})
for path, classification in SOURCES:
    data = data.append(build_data_frame(path, classification))

data = data.reindex(numpy.random.permutation(data.index))



In [8]:
pipeline = Pipeline([
    ('count_vectorizer',   CountVectorizer(ngram_range=(1, 2))),
    ('classifier',         MultinomialNB())
])

In [9]:
k_fold = KFold(n=len(data), n_folds=6)
scores = []
confusion = numpy.array([[0, 0], [0, 0]])
for train_indices, test_indices in k_fold:
    train_text = data.iloc[train_indices]['text'].values
    train_y = data.iloc[train_indices]['class'].values.astype(str)

    test_text = data.iloc[test_indices]['text'].values
    test_y = data.iloc[test_indices]['class'].values.astype(str)

    pipeline.fit(train_text, train_y)
    predictions = pipeline.predict(test_text)

    confusion += confusion_matrix(test_y, predictions)
    score = f1_score(test_y, predictions, pos_label=SPAM)
    scores.append(score)

In [39]:
print('Total emails classified:', len(data))
print('Score:', sum(scores)/len(scores))
print('Confusion matrix:')
print(confusion)

Total emails classified: 58910
Score: 0.978251240858
Confusion matrix:
[[23459    80]
 [ 1429 33942]]


Total emails classified: 58910
Score: 0.978251240858
Confusion matrix:
[[23459    80]
 [ 1429 33942]]

In [10]:
# Some experiments from our first attempts. 
# data
# print(len(data['text']))
# data.groupby('class').describe()

#### Q1.2.1 The vectorizer used in Zac Stewart's code is a CountVectorizer with unigrams and bigrams. Report the number of unigrams and bigrams used in this model.

In [40]:
# Retreive the count vectorizer used in the model. 
p=pipeline.get_params()
CountV=p['count_vectorizer']
#Access features: 
uni_bi_grams = CountV.get_feature_names()
print("There are %d unigrams and bigrams, used in this model. " %len(uni_bi_grams))

There are 3571898 unigrams and bigrams, used in this model. 


#### Q1.2.2 What are the 50 most frequent unigrams and bigrams in the dataset?

After long research, trial and error, and lots of stack overflow reading, We have this. 
Following the sturcture of the pipeline, as well as it's components, proved uncomfortable and 
counter intuitive, possibly on account of the amount of new information and functions. 
The structure follows a logic, and items are sorted in the same order, but can have no 
corresponding fields. For example, as seen below, we get the feature names from the count vector, 
but the feature count from the classifier. 

In [46]:
#We get the feature names from from the count vector as a list, and lists for feature count's, 
#with the index (0/1) specifing class. Zipping them together gives us a list of (feature, count in spam, count in ham)
def most_freq_feat(classifier, count_vector, n=50):
    index = 0
    features_c1_c2_count = []

    for feat, c1, c2 in zip(count_vector.get_feature_names(), classifier.feature_count_[0], classifier.feature_count_[1]):
        features_c1_c2_count.append((feat, c1 + c2))
        index+=1

    for i in sorted(features_c1_c2_count, key = lambda x: x[1], reverse=True)[:n]:     
        print(i)
    

    
most_freq_feat(p['classifier'], p['count_vectorizer'], n=10)

('font', 496308.0)
('3d', 477648.0)
('the', 440039.0)
('br', 422254.0)
('td', 345149.0)
('to', 322600.0)
('and', 256449.0)
('of', 231818.0)
('nbsp', 228765.0)
('http', 203133.0)


#### What are the 50 most frequent unigrams and bigrams per class (ham and spam)?

In [42]:
#Create a list of feature name and amount of occurrences in each class. 
#Sort according to different class counter to get occurrences per class. 
def most_occurring_feat_per_class(classifier, count_vector, n=50):
    index = 0
    features_c1_c2_count = []

    for feat, c1, c2 in zip(count_vector.get_feature_names(), classifier.feature_count_[0], classifier.feature_count_[1]):
        features_c1_c2_count.append((feat, c1, c2))
        index+=1

    print("%d most occurring features in class spam: " %n )    
    for i in sorted(features_c1_c2_count, key = lambda x: x[1], reverse=True)[:n]:     
        print(i)

    print("%d most occurring features in class ham: " %n )    
    for i in sorted(features_c1_c2_count, key = lambda x: x[2], reverse=True)[:n]:     
        print(i)


    
most_occurring_feat_per_class(p['classifier'], p['count_vectorizer'], n=50)

50 most occurring features in class spam: 
('the', 228461.0, 211578.0)
('to', 155374.0, 167226.0)
('and', 104175.0, 152274.0)
('of', 97814.0, 134004.0)
('in', 76007.0, 97121.0)
('com', 59545.0, 102710.0)
('for', 58849.0, 68774.0)
('enron', 57904.0, 3.0)
('is', 49909.0, 73549.0)
('on', 49315.0, 35525.0)
('http', 49090.0, 154043.0)
('that', 46778.0, 40383.0)
('you', 42453.0, 90704.0)
('td', 37809.0, 307340.0)
('this', 36165.0, 70785.0)
('it', 34585.0, 32132.0)
('with', 33303.0, 38826.0)
('be', 32529.0, 38583.0)
('ect', 31955.0, 117.0)
('20', 30188.0, 158033.0)
('width', 29023.0, 134549.0)
('have', 28261.0, 27348.0)
('from', 27854.0, 35797.0)
('www', 27705.0, 71213.0)
('will', 27625.0, 25286.0)
('as', 27477.0, 32776.0)
('we', 27189.0, 35266.0)
('at', 26653.0, 21809.0)
('http www', 26154.0, 63559.0)
('by', 24478.0, 22986.0)
('are', 24394.0, 33775.0)
('3d', 24097.0, 453551.0)
('of the', 22285.0, 23869.0)
('font', 21956.0, 474352.0)
('or', 20949.0, 41556.0)
('tr', 19165.0, 171177.0)
('if', 1

Output: 
    
    
50 most occurring features in class spam: 
('the', 228461.0, 211578.0)
('to', 155374.0, 167226.0)
('and', 104175.0, 152274.0)
('of', 97814.0, 134004.0)
('in', 76007.0, 97121.0)
('com', 59545.0, 102710.0)
('for', 58849.0, 68774.0)
('enron', 57904.0, 3.0)
('is', 49909.0, 73549.0)
('on', 49315.0, 35525.0)
('http', 49090.0, 154043.0)
('that', 46778.0, 40383.0)
('you', 42453.0, 90704.0)
('td', 37809.0, 307340.0)
('this', 36165.0, 70785.0)
('it', 34585.0, 32132.0)
('with', 33303.0, 38826.0)
('be', 32529.0, 38583.0)
('ect', 31955.0, 117.0)
('20', 30188.0, 158033.0)
('width', 29023.0, 134549.0)
('have', 28261.0, 27348.0)
('from', 27854.0, 35797.0)
('www', 27705.0, 71213.0)
('will', 27625.0, 25286.0)
('as', 27477.0, 32776.0)
('we', 27189.0, 35266.0)
('at', 26653.0, 21809.0)
('http www', 26154.0, 63559.0)
('by', 24478.0, 22986.0)
('are', 24394.0, 33775.0)
('3d', 24097.0, 453551.0)
('of the', 22285.0, 23869.0)
('font', 21956.0, 474352.0)
('or', 20949.0, 41556.0)
('tr', 19165.0, 171177.0)
('if', 18846.0, 20276.0)
('not', 18503.0, 28425.0)
('in the', 18200.0, 19452.0)
('your', 18147.0, 62630.0)
('br', 17243.0, 405011.0)
('09', 16493.0, 8327.0)
('height', 16276.0, 84761.0)
('src', 15728.0, 42217.0)
('gif', 15573.0, 31730.0)
('an', 15558.0, 20137.0)
('img', 15513.0, 44434.0)
('hou', 15114.0, 1859.0)
('has', 14987.0, 14681.0)
('10', 14958.0, 17121.0)
50 most occurring features in class ham: 
('font', 21956.0, 474352.0)
('3d', 24097.0, 453551.0)
('br', 17243.0, 405011.0)
('td', 37809.0, 307340.0)
('nbsp', 8072.0, 220693.0)
('the', 228461.0, 211578.0)
('size', 10499.0, 179557.0)
('tr', 19165.0, 171177.0)
('to', 155374.0, 167226.0)
('20', 30188.0, 158033.0)
('http', 49090.0, 154043.0)
('and', 104175.0, 152274.0)
('width', 29023.0, 134549.0)
('of', 97814.0, 134004.0)
('br br', 1173.0, 129623.0)
('nbsp nbsp', 1944.0, 123481.0)
('face', 7433.0, 121165.0)
('style', 2919.0, 109149.0)
('span', 968.0, 105849.0)
('align', 3430.0, 103625.0)
('border', 12265.0, 103592.0)
('com', 59545.0, 102710.0)
('in', 76007.0, 97121.0)
('color', 4859.0, 96178.0)
('html', 6217.0, 94434.0)
('font size', 3318.0, 91970.0)
('you', 42453.0, 90704.0)
('href', 13826.0, 89630.0)
('height', 16276.0, 84761.0)
('td tr', 8623.0, 82513.0)
('font face', 5871.0, 80826.0)
('content', 2404.0, 74778.0)
('td td', 8285.0, 74165.0)
('is', 49909.0, 73549.0)
('www', 27705.0, 71213.0)
('this', 36165.0, 70785.0)
('div', 1543.0, 70121.0)
('tr td', 6459.0, 69915.0)
('for', 58849.0, 68774.0)
('3d http', 4550.0, 65769.0)
('http www', 26154.0, 63559.0)
('style 3d', 360.0, 63308.0)
('arial', 5552.0, 62961.0)
('your', 18147.0, 62630.0)
('table', 11058.0, 62371.0)
('center', 3200.0, 58640.0)
('text', 3000.0, 53398.0)
('font td', 3791.0, 53000.0)
('tr tr', 3578.0, 52250.0)
('href 3d', 1499.0, 50821.0)

#### Q1.2.4 List the 20 most useful features in the Naive Bayes classifier to distinguish between spam and ham (20 features for each class). 

In [49]:
#Since each features coefficient links it to it's class, and smaller coefficients classify spam and larger ham, 
#we sort according to coefficient, once normaly and once reversed, to get most informative features. 
def most_informative_feature_for_binary_classification(vectorizer, classifier, n=20):
    class_labels = classifier.classes_
    feature_names = vectorizer.get_feature_names()
    topn_class1 = sorted(zip(classifier.coef_[0], feature_names))[:n]
    topn_class2 = sorted(zip(classifier.coef_[0], feature_names))[-n:]
    
    counter=0
    for coef, feat in topn_class1:
        print(class_labels[0], coef, feat)
        if counter==20: 
            break
    print()
    
    counter=0
    for coef, feat in reversed(topn_class2):
        print(class_labels[1], coef, feat)
        counter+=1
        if counter==n: 
            break

most_informative_feature_for_binary_classification(p['count_vectorizer'], p['classifier'], n=20)

ham -17.3273410308 00 005
ham -17.3273410308 00 00am
ham -17.3273410308 00 00pm
ham -17.3273410308 00 016
ham -17.3273410308 00 025
ham -17.3273410308 00 02pm
ham -17.3273410308 00 0392a0
ham -17.3273410308 00 048
ham -17.3273410308 00 04pm
ham -17.3273410308 00 0700
ham -17.3273410308 00 076
ham -17.3273410308 00 079
ham -17.3273410308 00 09may
ham -17.3273410308 00 09withdrawal
ham -17.3273410308 00 104
ham -17.3273410308 00 1104
ham -17.3273410308 00 113
ham -17.3273410308 00 118
ham -17.3273410308 00 1200
ham -17.3273410308 00 1206

spam -4.25763398155 font
spam -4.30247582511 3d
spam -4.41566905552 br
spam -4.69162787147 td
spam -5.02280862515 nbsp
spam -5.06498730061 the
spam -5.2290874763 size
spam -5.27688180104 tr
spam -5.30023358095 to
spam -5.35677555206 20
spam -5.38234747592 http
spam -5.39389765509 and
spam -5.51764987464 width
spam -5.52170863913 of
spam -5.55494779986 br br
spam -5.60349035536 nbsp nbsp
spam -5.62242424559 face
spam -5.72686266882 style
spam -5.75756275

Output: 
    
ham -17.3273410308 00 005
ham -17.3273410308 00 00am
ham -17.3273410308 00 00pm
ham -17.3273410308 00 016
ham -17.3273410308 00 025
ham -17.3273410308 00 02pm
ham -17.3273410308 00 0392a0
ham -17.3273410308 00 048
ham -17.3273410308 00 04pm
ham -17.3273410308 00 0700
ham -17.3273410308 00 076
ham -17.3273410308 00 079
ham -17.3273410308 00 09may
ham -17.3273410308 00 09withdrawal
ham -17.3273410308 00 104
ham -17.3273410308 00 1104
ham -17.3273410308 00 113
ham -17.3273410308 00 118
ham -17.3273410308 00 1200
ham -17.3273410308 00 1206

spam -4.25763398155 font
spam -4.30247582511 3d
spam -4.41566905552 br
spam -4.69162787147 td
spam -5.02280862515 nbsp
spam -5.06498730061 the
spam -5.2290874763 size
spam -5.27688180104 tr
spam -5.30023358095 to
spam -5.35677555206 20
spam -5.38234747592 http
spam -5.39389765509 and
spam -5.51764987464 width
spam -5.52170863913 of
spam -5.55494779986 br br
spam -5.60349035536 nbsp nbsp
spam -5.62242424559 face
spam -5.72686266882 style
spam -5.75756275423 span
spam -5.77879748822 align

#### Q1.2.5 There seems to be an imbalance in the length of spam and ham messages (see the plot in the attached notebook). We want to add a feature based on the number of words in the message in the text representation. Should the length attribute be normalized before fitting the Naive Bayes classifier? (See Sklearn pre-processing for examples.) Do you expect Logistic Regression to perform better with the new feature? Explain.

As can clearly be seen in below table, there are differences, and it does seem likely that these can shed some light as to the nature of a text. Looking into the following article:http://cs229.stanford.edu/proj2013/ShiraniMehr-SMSSpamDetectionUsingMachineLearningApproach.pdf, seems to confirm this suspision, concerning the multinomialNB. We will attempt some trial and error in the next question to verify this notion, and results will show improvment, in the normalized case. 

We expected the improvment to be more drastic with the LR, but were dissapointed to see it was not. 


<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right">
      <th></th>
      <th></th>
      <th>len</th>
    </tr>
    <tr>
      <th>class</th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th rowspan="8" valign="top">ham</th>
      <th>count</th>
      <td>21889.000000</td>
    </tr>
    <tr>
      <th>mean</th>
      <td>363.062589</td>
    </tr>
    <tr>
      <th>std</th>
      <td>1253.975002</td>
    </tr>
    <tr>
      <th>min</th>
      <td>1.000000</td>
    </tr>
    <tr>
      <th>25%</th>
      <td>59.000000</td>
    </tr>
    <tr>
      <th>50%</th>
      <td>149.000000</td>
    </tr>
    <tr>
      <th>75%</th>
      <td>318.000000</td>
    </tr>
    <tr>
      <th>max</th>
      <td>50295.000000</td>
    </tr>
    <tr>
      <th rowspan="8" valign="top">spam</th>
      <th>count</th>
      <td>10500.000000</td>
    </tr>
    <tr>
      <th>mean</th>
      <td>625.010000</td>
    </tr>
    <tr>
      <th>std</th>
      <td>914.769248</td>
    </tr>
    <tr>
      <th>min</th>
      <td>0.000000</td>
    </tr>
    <tr>
      <th>25%</th>
      <td>151.000000</td>
    </tr>
    <tr>
      <th>50%</th>
      <td>327.500000</td>
    </tr>
    <tr>
      <th>75%</th>
      <td>666.000000</td>
    </tr>
    <tr>
      <th>max</th>
      <td>16955.000000</td>
    </tr>
  </tbody>
</table>

#### Q1.2.6 Add the document length as a feature to the model. Does this new feature help? Use Sklearn FeatureUnion to combine the output of different vectorizers into a single vector. See for example Pipelines of feature unions by Zac Stewart, and Feature Union with Heterogeneous Data Sources from SkLearn's documentation.


In [38]:
# we've created a length feature extractor with the fit and transform functions.  
from sklearn.base import BaseEstimator, TransformerMixin    
class TextStats(BaseEstimator, TransformerMixin):
    """Extract features from each document for DictVectorizer"""

    def fit(self, x, y=None):
        return self

    def transform(self, posts):
        return [{'length': len(text)} for text in posts]

#        len(nltk.tokenize.word_tokenize(text))
#        return [{'length': len(text)} for text in posts]


In [54]:
#A new pipeline, adding theh length as a feature with normalization. 
from sklearn.pipeline import FeatureUnion
from sklearn import preprocessing
pipeline_w_l = Pipeline([
  ('features', FeatureUnion([
        ('body_stats', Pipeline([
                ('stats', TextStats()),  # returns a list of dicts
                ('vect', DictVectorizer()),  # list of dicts -> feature matrix
                ('scale', preprocessing.Normalizer()),              

                                ])),
        ('count_vectorizer',   CountVectorizer(ngram_range=(1, 2))),   
                            ])),
  ('classifier', MultinomialNB())
])


In [55]:
k_fold = KFold(n=len(data), n_folds=6)
scores = []
confusion = numpy.array([[0, 0], [0, 0]])
for train_indices, test_indices in k_fold:
    train_text = data.iloc[train_indices]['text'].values
    train_y = data.iloc[train_indices]['class'].values.astype(str)

    test_text = data.iloc[test_indices]['text'].values
    test_y = data.iloc[test_indices]['class'].values.astype(str)

    pipeline_w_l.fit(train_text, train_y)
    predictions = pipeline_w_l.predict(test_text)

    confusion += confusion_matrix(test_y, predictions)
    score = f1_score(test_y, predictions, pos_label=SPAM)
    scores.append(score)
    

print('Total emails classified:', len(data))
print('Score:', sum(scores)/len(scores))
print('Confusion matrix:')
print(confusion)

Total emails classified: 58910
Score: 0.978131441426
Confusion matrix:
[[23462    77]
 [ 1440 33931]]


Since each run takes ~Iternity (give or take), the results of the above run, with length added as a feature and
normalizing using the preproccessing unit, and the multinomialNB, can be found here: 

Total emails classified: 58910
Score: 0.978131441426
Confusion matrix:
[[23462    77]
 [ 1440 33931]]
 
We can see that even though the overall score decreased slightly, the number of false spams decreased substantialy. 
Obviously, this feature does improve our ability to recognize ham (though confuses us more with spam).  

Compared to the original classifier from zac's notebook:

Total emails classified: 58910
Score: 0.978284726388
Confusion matrix:
[[23450    89]
 [ 1418 33953]]
 
 When running the same pipeline without normalization, near no difference was noticed. Interesting. 

In [59]:
from sklearn.linear_model import LogisticRegression


pipeline_w_l_lr = Pipeline([
        ('features', FeatureUnion([
            ('body_stats', Pipeline([
                ('stats', TextStats()),  # returns a list of dicts
                ('vect', DictVectorizer()),  # list of dicts -> feature matrix
                ('scale', preprocessing.Normalizer()),              
                                ])),
        ('count_vectorizer',   CountVectorizer(ngram_range=(1, 2))),   
                            ])),
        ('classifier',         LogisticRegression())
    ])


In [61]:
#Lets try running the same scheme (almost) with the logisticRegression classifier, 
#and hope it finishes running before midnight monday. 
n_folds = 4
k_fold = KFold(n=len(data), n_folds = n_folds)
scores = []
confusion = numpy.array([[0, 0], [0, 0]])
print("Training with %d folds" % n_folds)
for i, (train_indices, test_indices) in enumerate(k_fold):
    train_text = data.iloc[train_indices]['text'].values
    train_y = data.iloc[train_indices]['class'].values.astype(str)
    test_text = data.iloc[test_indices]['text'].values
    test_y = data.iloc[test_indices]['class'].values.astype(str)

    print("Training for fold %d" % i)
    pipeline_w_l_lr.fit(train_text, train_y)
    print("Testing for fold %d" % i)
    predictions = pipeline_w_l_lr.predict(test_text)

    confusion += confusion_matrix(test_y, predictions)
    score = f1_score(test_y, predictions, pos_label=SPAM)
    scores.append(score)

    print("Score for %d: %2.2f" % (i, score))
    print("Confusion matrix for %d: " % i)
    print(confusion)

print('Total emails classified:', len(data))
print('Score:', sum(scores)/len(scores))
print('Confusion matrix:')
print(confusion)

Training with 4 folds
Training for fold 0
Testing for fold 0
Score for 0: 0.99
Confusion matrix for 0: 
[[5849   57]
 [ 200 8622]]
Training for fold 1
Testing for fold 1
Score for 1: 0.99
Confusion matrix for 1: 
[[11671    96]
 [  397 17292]]
Training for fold 2
Testing for fold 2
Score for 2: 0.98
Confusion matrix for 2: 
[[17417   133]
 [  643 25990]]
Training for fold 3
Testing for fold 3
Score for 3: 0.99
Confusion matrix for 3: 
[[23373   166]
 [  850 34521]]
Total emails classified: 58910
Score: 0.985502200587
Confusion matrix:
[[23373   166]
 [  850 34521]]


In [ ]:
                                                                                        
Training with 4 folds
Training for fold 0
Testing for fold 0
Score for 0: 0.99
Confusion matrix for 0: 
[[5849   57]
 [ 200 8622]]
Training for fold 1
Testing for fold 1
Score for 1: 0.99
Confusion matrix for 1: 
[[11671    96]
 [  397 17292]]
Training for fold 2
Testing for fold 2
Score for 2: 0.98
Confusion matrix for 2: 
[[17417   133]
 [  643 25990]]
Training for fold 3
Testing for fold 3
Score for 3: 0.99
Confusion matrix for 3: 
[[23373   166]
 [  850 34521]]
Total emails classified: 58910
Score: 0.985502200587
Confusion matrix:
[[23373   166]
 [  850 34521]]



### Q1.3 SMS Spam Dataset

#### Q1.3.1 Test the classifier trained on email data in 1.2 on the SMS data.
We are running the original pipeline trained by Zac in Q1.2.1, on the sms data. 

In [17]:

messages1 = pd.read_csv('./data/SMSSpamCollection', sep='\t', quoting=csv.QUOTE_NONE,
                           names=["class", "text"])

test_text = messages1['text'].values
test_y = messages1['class'].values.astype(str)

predictions = pipeline.predict(test_text)

confusion = numpy.array([[0, 0], [0, 0]])
confusion += confusion_matrix(test_y, predictions)
score = f1_score(test_y, predictions, pos_label=SPAM)
scores.append(score)

print('Total smss classified:', len(messages1))
print('Score:', sum(scores)/len(scores))
print('Confusion matrix:')
print(confusion)

Total smss classified: 5574
Score: 0.87226181865
Confusion matrix:
[[4274  553]
 [ 572  175]]


After running the original classifier from Zac's tutorial, we can see that althogh it does not do as well
on the new database, it does pretty well. These results fit our expectations. Spam does carry similar features 
over sms, but still varies from the original data set. 

Total smss classified: 5574
Score: 0.87226181865
Confusion matrix:
[[4274  553]
 [ 572  175]]

#### Q1.3.2 Test the SMS classifier trained on the email data

We'll take the SMS classifier from Radim's notebook, and run in with the same code Zac used to run his own.
Hopefully, this will assure as much symmetry as possible, and render our results as informative as possible. 
For this purpose we transformed Rare's (short for RNDr. Radim Řehůřek, Ph.D.) code into a pipeline, and the
ran Zac's code on the same data, with a different pipeline. 

For reference, this code: 

 bow_transformer = CountVectorizer(analyzer=split_into_lemmas).fit(data['text'])
 messages_bow = bow_transformer.transform(data['text'])
 tfidf_transformer = TfidfTransformer().fit(messages_bow)
 messages_tfidf = tfidf_transformer.transform(messages_bow)
 spam_detector = MultinomialNB().fit(messages_tfidf, data['class'])
 all_predictions = spam_detector.predict(messages_tfidf)


Was basically reduced to this code: 

rare_pipe=Pipeline([
            ('CV', CountVectorizer(analyzer=split_into_lemmas)),
            ('TFIDF',  TfidfTransformer()),
            ('Classifier', MultinomialNB())    
            
rare_pipe.fit(train_text, train_y)
predictions = rare_pipe.predict(test_text)

Neat!

Edit: Neat but doesn't work, which brings another word to mind!

In [ ]:
def split_into_tokens(message):
    return TextBlob(message).words


def split_into_lemmas(message):
    message = message.lower()
    words = TextBlob(message).words
    # for each word, take its "base form" = lemma 
    return [word.lemma for word in words]


bow_transformer = CountVectorizer(analyzer=split_into_lemmas).fit(data['text'])
messages_bow = bow_transformer.transform(data['text'])
tfidf_transformer = TfidfTransformer().fit(messages_bow)
messages_tfidf = tfidf_transformer.transform(messages_bow)
spam_detector = MultinomialNB().fit(messages_tfidf, data['class'])
all_predictions = spam_detector.predict(messages_tfidf)

print('accuracy', accuracy_score(data['class'], all_predictions))
print('confusion matrix\n', confusion_matrix(data['class'], all_predictions))
print('(row=expected, col=predicted)')


In [ ]:
# # messages = pandas.read_csv('/home/noam/Dropbox/pythonWorkspase/NLP_HW2/data_science_python/data/SMSSpamCollection', sep='\t', quoting=csv.QUOTE_NONE,
# #                            names=["label", "message"])

# def split_into_tokens(message):
#     return TextBlob(message).words


# def split_into_lemmas(message):
#     message = message.lower()
#     words = TextBlob(message).words
#     # for each word, take its "base form" = lemma 
#     return [word.lemma for word in words]


# rare_pipe=Pipeline([
#             ('CV', CountVectorizer(analyzer=split_into_lemmas)),
#             ('TFIDF',  TfidfTransformer()),
#             ('Classifier', MultinomialNB())    
#                    ])

# k_fold = KFold(n=len(data), n_folds=6)
# scores = []
# confusion = numpy.array([[0, 0], [0, 0]])
# i=0
# for train_indices, test_indices in k_fold:
#     print("In fold %d" %i)
#     i+=1
#     train_text = data.iloc[train_indices]['text'].values
#     train_y = data.iloc[train_indices]['class'].values.astype(str)

#     test_text = data.iloc[test_indices]['text'].values
#     test_y = data.iloc[test_indices]['class'].values.astype(str)

#     rare_pipe.fit(train_text, train_y)
#     predictions = rare_pipe.predict(test_text)

#     confusion += confusion_matrix(test_y, predictions)
#     score = f1_score(test_y, predictions, pos_label=SPAM)
#     scores.append(score)
    

# # print('accuracy', accuracy_score(messages['label'], all_predictions))
# # print('confusion matrix\n', confusion_matrix(data['class'], all_predictions))
# # print('(row=expected, col=predicted)')

# print('Total emails classified:', len(data))
# print('Score:', sum(scores)/len(scores))
# print('Confusion matrix:')
# print(confusion)

In [ ]:
CountV.get_stop_words()
CountV.get_params()['analyzer']

Sms classifier: 

accuracy 0.969501255831
confusion matrix
 [[4827    0]
 [ 170  577]]
(row=expected, col=predicted)

In [48]:
#ow_transformer = CountVectorizer(analyzer=split_into_lemmas).fit(data['text'])
#messages_bow = bow_transformer.transform(data['text'])
#tfidf_transformer = TfidfTransformer().fit(messages_bow)
#messages_tfidf = tfidf_transformer.transform(messages_bow)
#spam_detector = MultinomialNB().fit(messages_tfidf, data['class'])
all_predictions = spam_detector.predict(messages_tfidf)

print('accuracy', accuracy_score(data['class'], all_predictions))
print('confusion matrix\n', confusion_matrix(data['class'], all_predictions))
print('(row=expected, col=predicted)')


accuracy 0.967747411305
confusion matrix
 [[22495  1044]
 [  856 34515]]
(row=expected, col=predicted)


#### Q1.3.3 Measure the vocabulary and bigram mismatch between the 2 datasets: that is, find how many features are shared between the 2 datasets, appear only in one and only in the other.


In [13]:
#data - the dataframe defined in the begging of 1.2
#We will begin by gathering the features collected by the countVectorizer defined in rare's notebook. 
bow_transformer = CountVectorizer(analyzer=split_into_lemmas).fit(data['text'])
rare_feats = bow_transformer.get_feature_names()

In [19]:
#Now we will gather the features from Zac's vectorizer:
zacs_bow=CountVectorizer(ngram_range=(1, 2)).fit(data['text'])
zac_bow_words=zacs_bow.get_feature_names()

In [21]:
#Computing the intersection of the 2 sets: 
intersection = set(rare_feats).intersection(zac_bow_words) 

In [40]:
#The unique elements of each group: 
only_in_zac=list(set(zac_bow_words)-set(intersection))
only_in_rare=list(set(rare_feats)-set(intersection))

In [46]:
len_o_z = len(only_in_zac)
len_o_r = len(only_in_rare)
sum_o = len_o_z + len_o_r 
per_z = len_o_z/sum_o
per_r = len_o_r/sum_o
print("There are %d features in common, between both groups." %len(intersection))
print("There are %d unique features in Zac's model." % len_o_z)
print("There are %d unique features in Zac's model." %len_o_r)
print("Together, both model's have %d features. " %sum_o )
print("Clearly, Zac's model holds %f of all features. " % per_z)
print("Clearly, Rare's model holds %f of all features. " % per_r)
print("Zac's model is much larger, on account of using bigrams and not stemming. ")

There are 283869 features in common, between both groups.
There are 3732081 unique features in Zac's model.
There are 354145 unique features in Zac's model.
Together, both model's have 4086226 features. 
Clearly, Zac's model holds 0.913332 of all features. 
Clearly, Rare's model holds 0.086668 of all features. 
Zac's model is much larger, on account of using bigrams and not stemming. 


There are 283869 features in common, between both groups.
There are 3732081 unique features in Zac's model.
There are 354145 unique features in Zac's model.
Together, both model's have 4086226 features. 
Clearly, Zac's model holds 0.913332 of all features. 
Clearly, Rare's model holds 0.086668 of all features. 
Zac's model is much larger, on account of using bigrams and not stemming. 

### Q2. Named Entity Recognition
    Features:
            we are intrested in creating a vectorized obejct from our data set, which will take into consideration the
            (1) word-form (2)the POS of the word (3) ORT, (4) perfix1 ,(5) perfix2, (6) perfix3, (7) suffix1 
            (8) suffix2, (9) suffix3

first let's load and split our data set to test and train sets

In [4]:
# Split data set to train and test data sets
train_sents = list(nltk.corpus.conll2002.iob_sents('esp.train'))
test_sents = list(nltk.corpus.conll2002.iob_sents('esp.testb'))

 we've noticed that our data is built as a list of sentences, all of which are constructed from a list of tripules in the following foramt:

In [5]:
train_sents[0]

[('Melbourne', 'NP', 'B-LOC'),
 ('(', 'Fpa', 'O'),
 ('Australia', 'NP', 'B-LOC'),
 (')', 'Fpt', 'O'),
 (',', 'Fc', 'O'),
 ('25', 'Z', 'O'),
 ('may', 'NC', 'O'),
 ('(', 'Fpa', 'O'),
 ('EFE', 'NC', 'B-ORG'),
 (')', 'Fpt', 'O'),
 ('.', 'Fp', 'O')]

<div style='padding: 10px;'><code>[  [(&lt;WORD&gt;, &lt;POS&gt;, &lt;CLASS&gt;),....],<br />&nbsp;....[]&nbsp;...<br/>]</code>
</div><br/>
We would like to add another features, and will do that in a manner simmilar to the one being done
in the <a href="http://nbviewer.ipython.org/github/tpeng/python-crfsuite/blob/master/examples/CoNLL%202002.ipynb" target="_blank">CoNLL Classification</a> we'll build out a dictionary with all the
wanted features and use <i><u>DictVectorizer</u></i> to get a vectorized representation of the word
according to it's features. 
In order to do that let's define the following function to extract the features from our data-set, first, let's deine a function to extract ortographic data based on the word:

In [6]:
def get_ort(word):
    if (re.match("http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", word) != None):
        return "URL"
    if (re.match("^-?\+?[0-9]+(.[0-9]+)?$", word) != None):
        return "number"
    if (re.search(".*[0-9]+.*", word) != None):
        return "contains-digit"
    if (word.find("-") != -1):
        return "contains-hyphen"
    if word.isupper():
        return "all-capitals"
    if (re.match("^[A-Z].*", word) != None):
        return "capitalized"
    if (re.match("^[,;.-/!/?/*/+]+$", word) != None):
        return "punctuation"
    return "regular"

This function will return a string value representing one ortographic feature for a word. For simplicity any word has a single orthograpic feature, and if it doesn't contain any we will say the word has a regular stracture. Now based on this function let's define a function to extract the features from a given word:

In [7]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    features = {
        'WORD-FROM': word.lower(),
        'POS': postag,
        'ORT': get_ort(word),
        'PREFIX1': word.lower()[:1],
        'PREFIX2': word.lower()[:2],
        'PREFIX3': word.lower()[:3],
        'SUFFIX1': word.lower()[-1:],
        'SUFFIX2': word.lower()[-2:],
        'SUFFIX3': word.lower()[-3:]}
    return features

This function will extract the wanted features for each index in te sentence array provided to it. The features will be sent as a dictionary, which is the prefered format of the <b><u>DictVectorizer</u></b> which will later convert is to a one-hot vector.
<br />Let's also define a function to easly extract the NER tag from a sentence:


In [8]:
def word2label(sent, i):
    return sent[i][2]

We can now construct our model's pipeline based on the <b>DictVectorizer</b> and the <b>Logistic Regression Classifier</b>:

In [39]:
def build_pipeline():
    pipeline = Pipeline([
        ('vectorize', DictVectorizer(sparse=True)),
        ('classify', LogisticRegression())
    ])
    return pipeline

We will first use the Spanish data-set for training and testing, let's define our sets:

In [36]:
train_sents = list(nltk.corpus.conll2002.iob_sents('esp.train'))
test_sents = list(nltk.corpus.conll2002.iob_sents('esp.testb'))

Now this data set is pretty big, (we acturally have tried training our pipline with simple array - which abviusly failed badly due to lack of memory) in order to overcome this problem, we'll load the data-set to a numpy arrays, and along the way will already build it in a format which will be easer for us to extract our features from. <br />
First let's re-use the progress function from the SPAM note-book, so we won't be bored while the data is processed:

In [11]:
def progress(i, end_val, bar_length=50):
    '''
    Print a progress bar of the form: Percent: [#####      ]
    i is the current progress value expected in a range [0..end_val]
    bar_length is the width of the progress bar on the screen.
    '''
    percent = float(i) / end_val
    hashes = '#' * int(round(percent * bar_length))
    spaces = ' ' * (bar_length - len(hashes))
    sys.stdout.write("\rPercent: [{0}] {1}%".format(hashes + spaces, int(round(percent * 100))))
    sys.stdout.flush()

Now we will write a function to build our data in a numpy DataFrame while extracting out our features:

In [12]:
def build_data(data, feature_extractor=(lambda sent, i: word2features(sent, i))):
    df = DataFrame({'features': [], 'class': []})
    print("Starting To Build Data.")
    for i, sent in enumerate(data):
        data_frame, nrows = build_data_frame(i, len(data), sent, feature_extractor)
        df = df.append(data_frame)
    print("\nDone!")
    return df

This will return a data frame with 'features' holding our dictionary of features (based on our feature extractor - the default one will be the one defined earlier) and 'class' will hold the values to be learned by the classifier - in our case the NER tag.
and Here is the <i>build_data_frame</i> function:

In [13]:
def build_data_frame(l, len_data, sent, feature_extractor):
    rows = []
    index = []
    for i in range(len(sent)):
        rows.append({'features': feature_extractor(sent, i), 'class': word2label(sent, i)})
        index.append(sent)
    progress(l, len_data)
    data_frame = DataFrame(rows, index=index)
    return data_frame, len(rows)

Now we are redy for our training function, again we should keep in mind that our data-set it pretty big, so in order to easely work with our data we will split the data set using <b><u>KFold</u></b> simmilar to the way it's being done with the SPAM detector:

In [17]:
def train(data_sents=None, data_frame=None, n_folds=6):
    if data_frame is None and data_sents is None:
        raise Exception('No data was provided to train!')
    elif data_frame is None:
        data_frame = build_data(data_sents)

    k_fold = KFold(n=len(data_frame), n_folds=n_folds)
    pipeline = build_pipeline()

    print("Training with %d folds" % n_folds)
    for i, (train_indices, test_indices) in enumerate(k_fold):
        x_train = data_frame.iloc[train_indices]['features'].values
        y_train = data_frame.iloc[train_indices]['class'].values.astype(str)
        if (q2Verbose):
            print("Training for fold %d" % i)
        pipeline.fit(x_train, y_train)
    print('Total classified:', len(data_frame))
    return pipeline, data_frame

If we're already at-it let's also write another feature extractor, one which will also take into consideration the previouse and next words:

In [27]:
# Now we can play a bit with the features
def word2features2(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    fetures = {
        'WORD-FROM': word.lower(),
        'POS': postag,
        'ORT': get_ort(word),
        'PREFIX1': word.lower()[:1],
        'PREFIX2': word.lower()[:2],
        'PREFIX3': word.lower()[:3],
        'SUFFIX1': word.lower()[-1:],
        'SUFFIX2': word.lower()[-2:],
        'SUFFIX3': word.lower()[-3:]}

    if i < (len(sent) - 1):
        fetures['NEXT_WORD_FORM'] = sent[i + 1][0]
        fetures['NEXT_POS'] = sent[i + 1][1]
    else:
        fetures['NEXT_WORD_FORM'] = '*'
        fetures['NEXT_POS'] = '*'

    if i > 0:
        fetures['PREV_WORD_FORM'] = sent[i - 1][0]
        fetures['PREV_POS'] = sent[i - 1][1]
    else:
        fetures['PREV_WORD_FORM'] = '*'
        fetures['PREV_POS'] = '*'

    return fetures

Here we have also added an ending, and beginning padding with '*' to mark words at a beginning of a sentence or words without followers. Now we are ready to load our data, and extract it's features. this might take a while (up to 10 minutes) <br />
<b>The first one is the default extractor (without next and prev word features)</b> we'll call it data1

In [37]:
data1 = build_data(train_sents)

Starting To Build Data.
Percent: [##################################################] 100%
Done!


<b>The second is the more advanced extractor (with next and prev word features)</b> we'll call it data2

In [41]:
data2 = build_data(train_sents, word2features2)

Starting To Build Data.
Percent: [##################################################] 100%
Done!


Now we can train both of our models (based on diffrent features) and see the results:

In [40]:
model1, data1 = train(None, data1)

Training with 6 folds
Training for fold 0
Training for fold 1
Training for fold 2
Training for fold 3
Training for fold 4
Training for fold 5
Total classified: 264715


In [44]:
model2, data2 = train(None, data2)

Training with 6 folds
Training for fold 0
Training for fold 1
Training for fold 2
Training for fold 3
Training for fold 4
Training for fold 5
Total classified: 264715


Let's test the outputs of both of the models, first we'll define those two helper function to extract data from the data-set

In [45]:
def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [word2label(sent, i) for i in range(len(sent))]

In [46]:
x_test = sum([sent2features(s) for s in test_sents], [])
y_test = sum([sent2labels(s) for s in test_sents], [])

This will build us two arrays of features and NER tags

In [50]:
print("testing model1..")
predictions1 = model1.predict(x_test)
score1 = f1_score(y_test, predictions1)
print("First model (without looking on previous and next word tags) scored %f " %score1 )

testing model1..
First model (without looking on previous and next word tags) scored 0.937961 


/home/noam/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:756: DeprecationWarning: The default `weighted` averaging is deprecated, and from version 0.18, use of precision, recall or F-score with multiclass or multilabel data or pos_label=None will result in an exception. Please set an explicit value for `average`, one of (None, 'micro', 'macro', 'weighted', 'samples'). In cross validation use, for instance, scoring="f1_weighted" instead of scoring="f1".
  sample_weight=sample_weight)


let's extract featues according to the <b>second feature extractor</b>:

In [51]:
def sent2features2(sent):
    return [word2features2(sent, i) for i in range(len(sent))]

x_test2 = sum([sent2features2(s) for s in test_sents], [])

In [52]:
print("testing model2..")
predictions2 = model2.predict(x_test2)
score2 = f1_score(y_test, predictions2)

testing model2..


/home/noam/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:756: DeprecationWarning: The default `weighted` averaging is deprecated, and from version 0.18, use of precision, recall or F-score with multiclass or multilabel data or pos_label=None will result in an exception. Please set an explicit value for `average`, one of (None, 'micro', 'macro', 'weighted', 'samples'). In cross validation use, for instance, scoring="f1_weighted" instead of scoring="f1".
  sample_weight=sample_weight)


We have used the f1 (AKA F_Masure) score to check our models, let's compare the outputs:

In [53]:
print("First model (without looking on previous and next word tags) scored %f " %score1 )
print("After adding to the feature extraction better features we were managed to score  %f" %score2 )

print ("Here is both confusions matrix of the first one:\n", confusion_matrix(y_test, predictions1))
print ("And Here is the second:\n", confusion_matrix(y_test, predictions2))

First model (without looking on previous and next word tags) scored 0.937961 
After adding to the feature extraction better features we were managed to score  0.958372
Here is both confusions matrix of the first one:
 [[  48    0    2    0    2    4    4    5    8]
 [   0   10    1    2    0    1    1    0    8]
 [  12    2   86    7    0    1    4    2    2]
 [   4    0    0   33    2    0    0   15    1]
 [   4    0    3    2    2    0    3    2    6]
 [   2    0    1    1    0    7    0    3   10]
 [   9    5    4    4    0    1   32    2   34]
 [   3    0    0    9    1    1    8   25    2]
 [   0    0    0    0    0    0    1    0 3257]]
And Here is the second:
 [[  47    0    2    6    3    3    3    1    8]
 [   0   11    3    0    0    1    2    1    5]
 [  13    2   87    4    1    1    0    3    5]
 [   4    0    2   41    0    0    1    6    1]
 [   1    0    2    0   10    0    2    2    5]
 [   2    0    1    0    1   10    2    0    8]
 [   3    0    3    0    0    5   59

We can now clearly see that adding the context to the feature extraction have added a 7% better f_masure score! now we can see if this is also the case with a diffrent language, let's load the dutch dataset:

In [ ]:
dutch_train_sents = list(nltk.corpus.conll2002.iob_sents('ned.train'))
dutch_test_sents_a = list(nltk.corpus.conll2002.iob_sents('ned.testa'))
dutch_test_sents_a = list(nltk.corpus.conll2002.iob_sents('ned.testb'))

Let's extract our features to numpy array:

In [ ]:
data_dutch = build_data(dutch_train_sents, word2features2)

In [ ]:
model_dutch, data_dutch = train(None, data_dutch)

Now let's test it wit testa

In [ ]:
x_test_dutch = sum([sent2features2(s) for s in dutch_test_sents_a], [])
y_test = sum([sent2labels(s) for s in dutch_test_sents_a], [])
predictions_dutcha = model_dutch.predict(x_test_dutch)
score_dutch1 = f1_score(y_test, predictions_dutcha)
print(score_dutch1)

Now let's add testb

In [ ]:
x_test_dutchb = sum([sent2features2(s) for s in dutch_test_sents_b], [])
y_testb = sum([sent2labels(s) for s in dutch_test_sents_b], [])
predictions_dutchb = model_dutch.predict(x_test_dutchb)
score_dutch2 = f1_score(y_test, predictions_dutchb)
print(score_dutch2

Let's calculate the average score:

In [ ]:
avg_dutch_score = (score_dutch2+score_dutch1)/2
print("Our NER tagger scored %f in dutch", % avg_dutch_score)

We can now see that our NER tagger had scored X in dutch, while scoring Y in Spanish. We guess this as a lot to do with spanish articles since they have unique atricles for locations and placed..